In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib.request 
import time
import re

#### Uji link web melalui chrome

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.instagram.com/infogresik/')

#### Akses web sampai page bawah

In [3]:
lenPage = driver.execute_script("window.scrollTo(0,document.body.scrollHeight); var lenPage = document.body.scrollHeight; return lenPage;")
match = False
while(match==False):
    lastCount = lenPage
    time.sleep(3)
    lenPage = driver.execute_script("window.scrollTo(0,document.body.scrollHeight); var lenPage = document.body.scrollHeight; return lenPage;")
    if lastCount == lenPage:
        match=True

#### Scrape postingan beserta caption

In [4]:
#create empty list
posts = []
capt_res = []
texts = []

links = driver.find_elements(by=By.TAG_NAME, value='a')
for link in links:
    post = link.get_attribute('href')
    if '/p/' in post:
        posts.append(post)
        
content = driver.page_source
soup = BeautifulSoup(content, features='html.parser')
for ph_name in soup.find_all(attrs='KL4Bh'):
    caption = ph_name.find('img', {'src':re.compile('https:')})
    if caption not in capt_res:
        capt_res.append(str(caption))
        
for text in capt_res:
    pisah = text.split('alt="')
    pisah2 = pisah[1].split('"')
    texts.append(pisah2[0])

#### Save to CSV

In [5]:
df = pd.DataFrame({'Postingan': posts, 'caption' : texts})
df.to_csv('instagram.csv', index=False, encoding='utf-8')

#### Download postingan agar berupa gambar bukan link

In [6]:
download_url = ''
for post in posts:
    driver.get(post)
    shortcode = driver.current_url.split("/")[-2]
    type = driver.find_element(by=By.XPATH, value='//meta[@property="og:type"]').get_attribute("content")
    if type == 'video':
        download_url = driver.find_element_by_xpath('//meta[@property="og:video"]').get_attribute("content")
        urllib.request.urlretrieve(download_url, "{}.mp4".format(shortcode))
    else:
        download_url = driver.find_element(by=By.XPATH, value='//meta[@property="og:image"]').get_attribute("content")
        urllib.request.urlretrieve(download_url, "{}.jpg".format(shortcode))